In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from numpy import asarray
import glob
import cv2
import tensorflow as tf
from tensorflow import keras

Import Model

In [2]:
UnWeighted_Model=keras.models.load_model('drive/MyDrive/CNN_91.57Acc_87.22')

Import Data

In [3]:
X_train=np.load('drive/MyDrive/X_train_strat.npy')

In [4]:
y_train=np.load('drive/MyDrive/y_train_strat.npy')

In [5]:
X_val=np.load('drive/MyDrive/X_val_strat.npy')

In [6]:
y_val=np.load('drive/MyDrive/y_val_strat.npy')

Get val data probabilities

In [7]:
y_proba=UnWeighted_Model.predict(X_val)

In [8]:
y_proba[1]

array([1.87001541e-01, 2.06485856e-03, 6.50142655e-02, 5.69395888e-06,
       2.40765674e-09, 3.12695047e-03, 8.90761893e-03, 5.98644692e-05,
       9.28697293e-04, 3.26755084e-03, 1.40106382e-21, 2.30899802e-03,
       5.62404156e-01, 1.20877512e-01, 2.80299364e-03, 1.99087546e-04,
       5.30674843e-05, 1.53191015e-02, 8.92879996e-07, 2.19788766e-04,
       1.39421183e-11, 1.09830246e-07, 3.47746137e-10, 9.22895538e-07,
       2.23131534e-02, 1.04928180e-03, 3.09432880e-06, 2.07079411e-03],
      dtype=float32)

In [9]:
# Turn one hot array into integers
y_val_1d=np.argmax(y_val, axis=1)
y_predictions=np.argmax(y_proba, axis=1)

In [10]:
y_dataframe=pd.DataFrame({'y_predictions': y_predictions,
                          'y_actual': y_val_1d
                          })

In [11]:
y_dataframe

,y_predictions,y_actual
0,25,25
1,12,12
2,8,2
3,23,23
4,13,14
...,...,...
8268,1,1
8269,13,13
8270,1,1
8271,1,1


In [12]:
y_proba_list=[]

for each in y_proba:
  y_proba_list.append(each)

In [ ]:
#y_dataframe['y_proba']=y_proba_list

In [13]:
#round to 5 decimals

rounded_proba_list=[]
for each_array in y_proba_list:
  new_array=[]
  for each_number in each_array:
    rounded_number=round(each_number, ndigits=5)
    new_array.append('{0:6f}'.format(rounded_number))
  rounded_proba_list.append(new_array)

In [14]:
y_dataframe['y_rounded_proba']=rounded_proba_list

In [15]:
y_dataframe.head(5)

,y_predictions,y_actual,y_rounded_proba
0,25,25,"[0.000290, 0.000050, 0.444450, 0.000000, 0.000..."
1,12,12,"[0.187000, 0.002060, 0.065010, 0.000010, 0.000..."
2,8,2,"[0.000000, 0.000050, 0.368030, 0.000210, 0.000..."
3,23,23,"[0.000000, 0.000000, 0.000000, 0.000000, 0.000..."
4,13,14,"[0.001320, 0.002030, 0.005430, 0.000550, 0.000..."


In [ ]:
#y_dataframe['y_rounded_proba'][15]

In [16]:
## locate incorrectly classified animals
incorrect_df=y_dataframe.loc[(y_dataframe['y_predictions'])!=(y_dataframe['y_actual'])]

In [17]:
incorrect_df

,y_predictions,y_actual,y_rounded_proba
2,8,2,"[0.000000, 0.000050, 0.368030, 0.000210, 0.000..."
4,13,14,"[0.001320, 0.002030, 0.005430, 0.000550, 0.000..."
6,12,13,"[0.089220, 0.001030, 0.009610, 0.000010, 0.000..."
8,13,12,"[0.195130, 0.004920, 0.005880, 0.000130, 0.000..."
15,2,16,"[0.000010, 0.000000, 0.990060, 0.000000, 0.000..."
...,...,...,...
8210,2,8,"[0.000000, 0.000010, 0.678840, 0.000000, 0.000..."
8227,8,2,"[0.000010, 0.003020, 0.128240, 0.004250, 0.000..."
8233,2,26,"[0.000000, 0.000010, 0.464550, 0.000600, 0.000..."
8234,8,16,"[0.000000, 0.000000, 0.018780, 0.000000, 0.000..."


In [18]:
## add columns for predicted probability and predicted probability of actual class
predicted_class_probability=[]
actual_class_probability=[]
for index, row in incorrect_df.iterrows():
  p=row['y_predictions']
  a=row['y_actual']
  pred_proba=row['y_rounded_proba'][p]
  act_proba=row['y_rounded_proba'][a]
  predicted_class_probability.append(pred_proba)
  actual_class_probability.append(act_proba)

In [19]:
incorrect_df['predicted_class_probability']=predicted_class_probability
incorrect_df['actual_class_probability']=actual_class_probability
incorrect_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,y_predictions,y_actual,y_rounded_proba,predicted_class_probability,actual_class_probability
2,8,2,"[0.000000, 0.000050, 0.368030, 0.000210, 0.000...",0.595060,0.368030
4,13,14,"[0.001320, 0.002030, 0.005430, 0.000550, 0.000...",0.456610,0.191310
6,12,13,"[0.089220, 0.001030, 0.009610, 0.000010, 0.000...",0.582400,0.215040
8,13,12,"[0.195130, 0.004920, 0.005880, 0.000130, 0.000...",0.394860,0.042490
15,2,16,"[0.000010, 0.000000, 0.990060, 0.000000, 0.000...",0.990060,0.004100
...,...,...,...,...,...
8210,2,8,"[0.000000, 0.000010, 0.678840, 0.000000, 0.000...",0.678840,0.008080
8227,8,2,"[0.000010, 0.003020, 0.128240, 0.004250, 0.000...",0.412930,0.128240
8233,2,26,"[0.000000, 0.000010, 0.464550, 0.000600, 0.000...",0.464550,0.347820
8234,8,16,"[0.000000, 0.000000, 0.018780, 0.000000, 0.000...",0.580390,0.376710


In [20]:
#find differences in predicted vs actual probability
incorrect_df['difference']=(incorrect_df['predicted_class_probability'].astype(float))-(incorrect_df['actual_class_probability'].astype(float))
incorrect_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,y_predictions,y_actual,y_rounded_proba,predicted_class_probability,actual_class_probability,difference
2,8,2,"[0.000000, 0.000050, 0.368030, 0.000210, 0.000...",0.595060,0.368030,0.22703
4,13,14,"[0.001320, 0.002030, 0.005430, 0.000550, 0.000...",0.456610,0.191310,0.26530
6,12,13,"[0.089220, 0.001030, 0.009610, 0.000010, 0.000...",0.582400,0.215040,0.36736
8,13,12,"[0.195130, 0.004920, 0.005880, 0.000130, 0.000...",0.394860,0.042490,0.35237
15,2,16,"[0.000010, 0.000000, 0.990060, 0.000000, 0.000...",0.990060,0.004100,0.98596
...,...,...,...,...,...,...
8210,2,8,"[0.000000, 0.000010, 0.678840, 0.000000, 0.000...",0.678840,0.008080,0.67076
8227,8,2,"[0.000010, 0.003020, 0.128240, 0.004250, 0.000...",0.412930,0.128240,0.28469
8233,2,26,"[0.000000, 0.000010, 0.464550, 0.000600, 0.000...",0.464550,0.347820,0.11673
8234,8,16,"[0.000000, 0.000000, 0.018780, 0.000000, 0.000...",0.580390,0.376710,0.20368


In [21]:
# ## get counts for each misclassified class
missed_counts=incorrect_df['y_actual'].value_counts()
missed_counts

8     146
13    133
2     131
9      65
16     62
25     58
12     50
17     46
14     43
5      42
1      40
26     34
0      29
6      28
19     21
18     18
15     15
22     14
24     13
27     13
20     12
7      10
11     10
21     10
23      7
4       4
3       2
10      1
Name: y_actual, dtype: int64

In [23]:
table13=incorrect_df.loc[incorrect_df['y_actual']==13]

In [24]:
table13

,y_predictions,y_actual,y_rounded_proba,predicted_class_probability,actual_class_probability,difference
6,12,13,"[0.089220, 0.001030, 0.009610, 0.000010, 0.000...",0.582400,0.215040,0.36736
124,9,13,"[0.001540, 0.000880, 0.008140, 0.045360, 0.000...",0.500800,0.300520,0.20028
182,12,13,"[0.007460, 0.000150, 0.000660, 0.000000, 0.000...",0.543480,0.259370,0.28411
360,12,13,"[0.004570, 0.000000, 0.000020, 0.000000, 0.000...",0.677490,0.312470,0.36502
392,17,13,"[0.000100, 0.000000, 0.000120, 0.000000, 0.000...",0.949240,0.046880,0.90236
...,...,...,...,...,...,...
7870,12,13,"[0.000040, 0.000000, 0.000140, 0.000000, 0.000...",0.998050,0.001150,0.99690
7981,12,13,"[0.000010, 0.000000, 0.000000, 0.000000, 0.000...",0.999660,0.000290,0.99937
8031,23,13,"[0.020930, 0.023410, 0.166830, 0.000850, 0.000...",0.302990,0.205110,0.09788
8088,9,13,"[0.000010, 0.000030, 0.000020, 0.000340, 0.000...",0.845300,0.147080,0.69822


In [ ]:
## find average difference
avg=(table13['difference'].sum())/len(table13['difference'])
avg

## 13: 0.52

0.5207695161290322

In [28]:
## find median difference
import statistics
med=statistics.median(table13['difference'])
med

## 13: 0.46

0.46371000000000007

Create Weights for underepresented classes

In [22]:
#Manually adjusting weights for only the poorly classified classes
adjusted_class_weights={
    0: 1.,
    1: 1.,
    2: 1.,
    3: 1.,
    4: 1.,
    5: 1.,
    6: 1.,
    7: 1.,
    8: 1.,
    9: 1.,
    10: 1.,
    11: 1.,
    12: 1.,
    13: 1.46,
    14: 1.,
    15: 1.,
    16: 1.,
    17: 1.,
    18: 1.,
    19: 1.,
    20: 1.,
    21: 1.,
    22: 1.,
    23: 1.,
    24: 1.,
    25: 1.,
    26: 1.,
    27: 1.}

Add weights to training

In [27]:
from tensorflow.keras import layers
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model4 = keras.models.Sequential()
model4.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(150,200,1),padding='same'))
model4.add(MaxPooling2D((2, 2),padding='same'))
model4.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.28))
model4.add(Conv2D(128, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.35))
model4.add(Conv2D(256, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model4.add(Dropout(0.40))
model4.add(Conv2D(512, (3, 3), activation='relu',padding='same'))          
model4.add(MaxPooling2D(pool_size=(6, 6),padding='same'))               ## changed from 4,4 to 6,6
model4.add(Dropout(0.40))
model4.add(Conv2D(256, (3, 3), activation='relu',padding='same'))               
model4.add(MaxPooling2D(pool_size=(6,6),padding='same'))                 ##changed from 4,4 to 6,6
model4.add(Dropout(0.40))
model4.add(Conv2D(256, (3, 3), activation='relu',padding='same'))      ## added conv, pooling layer           
model4.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model4.add(Dropout(0.35))
model4.add(Flatten())
model4.add(Dense(300, activation='relu'))               
model4.add(Dense(28, activation='softmax'))

In [28]:

model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
# callback to save best parameters
checkpoint=tf.keras.callbacks.ModelCheckpoint(
    filepath='drive/MyDrive/CNN4_2',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

In [30]:
model4.fit(X_train, y_train, batch_size=90, epochs=85, validation_data=(X_val, y_val), callbacks=[checkpoint])

Epoch 1/85
368/368 [==============================] - 33s 87ms/step - loss: 2.2465 - accuracy: 0.3153 - val_loss: 1.5713 - val_accuracy: 0.5192
Epoch 2/85
368/368 [==============================] - 30s 83ms/step - loss: 1.4414 - accuracy: 0.5690 - val_loss: 1.2144 - val_accuracy: 0.6317
Epoch 3/85
368/368 [==============================] - 31s 84ms/step - loss: 1.2421 - accuracy: 0.6218 - val_loss: 1.1089 - val_accuracy: 0.6571
Epoch 4/85
368/368 [==============================] - 31s 84ms/step - loss: 1.1186 - accuracy: 0.6570 - val_loss: 0.9506 - val_accuracy: 0.7074
Epoch 5/85
368/368 [==============================] - 31s 83ms/step - loss: 1.0405 - accuracy: 0.6838 - val_loss: 0.9338 - val_accuracy: 0.7149
Epoch 6/85
368/368 [==============================] - 31s 84ms/step - loss: 0.9722 - accuracy: 0.7039 - val_loss: 0.8492 - val_accuracy: 0.7399
Epoch 7/85
368/368 [==============================] - 31s 83ms/step - loss: 0.9272 - accuracy: 0.7161 - val_loss: 0.8048 - val_accuracy:

In [ ]:
## weighted all 1's                            Accuracy: 79   Val Accuracy: 79  after 29 epochs
## No weights                                  Accuracy: 79   Val Accuracy: 78  after 37 epochs
## No weights batch size 80                    Accuracy: 85   Val Accuracy: 81  after 30 epochs
## No weights batch size 90                    Accuracy: 88   Val Accuracy: 83.9  after 46 epochs?
## NW, BS 90, last conv2d 256                  Accuracy: 84   Val Accuracy: 83  after 26 epochs
## NW, bs 90, last conv2d 256, dense 400       Accuracy: 88   Val Accuracy: 85.29 after 53 epochs
## NW, bs 90, ', dense 300, 512 pool(4x4)      Accuracy: 86.9 Val Accuracy: 85.75 after 48 epochs

In [34]:
UnWeighted_Model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 200, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 100, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 100, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 50, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 38, 50, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 38, 50, 128)       7